#### Drop variables I do not need

In [ ]:
# ds_dropped = ds.drop({'AG1', 'AL', 'AL1', 'COD', 'CP', 'CWP', 'CZ', 'FRA', 'FRV', 'G11', 'G21', 'IWP', 'LHF', 'LWD', 'LWU', 'ME', 'QI', 'QQ', 'QQP', 'QQZ', 'QR', 'QS', 'QW', 'RF', 'RH', 'RU2', 'RZ', 'SAL', 'SF', 'SHF', 'SHSN0', 'SHSN3', 'SLO', 'SOL', 'SP', 'SRF', 'ST', 'ST2', 'SU', 'SW', 'SWD', 'TI1', 'TT', 'TTMAX', 'TTMIN', 'TTP', 'TTZ', 'UU', 'UUP', 'UUZ', 'UV', 'UVZ', 'VEG', 'VV', 'VVP', 'VVZ', 'WA1', 'WVP', 'ZN4', 'ZN5', 'ZN6', 'ZZ', 'ZZP'})
# ds_dropped

#### Rechunking

In [ ]:
# ds_dropped = ds_dropped.chunk({'TIME': 1})

#### Create mapper

In [ ]:
# mapper = gdfs.get_mapper('ICE.2000-2010.zarr')

#### Save data to Zarr file on Google Drive

In [ ]:
# ds_dropped.to_zarr(mapper, consolidated=True)

#### Open data from Zarr file from Google Drive

In [ ]:
# ds = xr.open_zarr(mapper, decode_coords=False) #, mask_and_scale=False) #, consolidated=True)

### Test opening tutorial data set, saving in Zarr and opening in Zarr

In [1]:
# gdfid_tutorial = '162zJ0ZyHaq9dkU4h3MdiZu1Vma8rPxkQ'
# gdfs_tutorial = gdrivefs.GoogleDriveFileSystem(root_file_id = gdfid_tutorial, token = 'cache')

In [2]:
# mapper_tutorial = gdfs_tutorial.get_mapper('air_temperature.zarr')

In [3]:
# ds_tutorial = xr.tutorial.open_dataset("air_temperature")

In [4]:
# ds_tutorial.to_zarr(mapper_tutorial)

In [5]:
# ds_tut_test = xr.open_zarr(mapper_tutorial)

In [6]:
# ds_tut_test

In [7]:
# ds_tut_test.air[0,0,0].load()

### Test rechunker

In [2]:
import xarray as xr
xr.set_options(display_style='text')
import zarr
import dask.array as dsa

In [3]:
ds = xr.tutorial.open_dataset("air_temperature")
# create initial chunk structure
ds = ds.chunk({'time': 100})
ds.air.encoding = {} # helps when writing to zarr
ds

<xarray.Dataset>
Dimensions:  (lat: 25, lon: 53, time: 2920)
Coordinates:
  * lat      (lat) float32 75.0 72.5 70.0 67.5 65.0 ... 25.0 22.5 20.0 17.5 15.0
  * lon      (lon) float32 200.0 202.5 205.0 207.5 ... 322.5 325.0 327.5 330.0
  * time     (time) datetime64[ns] 2013-01-01 ... 2014-12-31T18:00:00
Data variables:
    air      (time, lat, lon) float32 dask.array<chunksize=(100, 25, 53), meta=np.ndarray>
Attributes:
    Conventions:  COARDS
    title:        4x daily NMC reanalysis (1948)
    description:  Data is from NMC initialized reanalysis\n(4x/day).  These a...
    platform:     Model
    references:   http://www.esrl.noaa.gov/psd/data/gridded/data.ncep.reanaly...

In [4]:
ds.air.data

,Array,Chunk
Bytes,15.48 MB,530.00 kB
Shape,"(2920, 25, 53)","(100, 25, 53)"
Count,31 Tasks,30 Chunks
Type,float32,numpy.ndarray


In [20]:
! rm -rf *.zarr # clean up any existing temporary data
ds.to_zarr('air_temperature.zarr')

In [6]:
source_group = zarr.open('air_temperature.zarr')
print(source_group.tree())

/
 ├── air (2920, 25, 53) float32
 ├── lat (25,) float32
 ├── lon (53,) float32
 └── time (2920,) float32


In [21]:
type(source_group)

zarr.hierarchy.Group

In [7]:
source_array = source_group['air']
source_array.info

Name,/air
Type,zarr.core.Array
Data type,float32
Shape,"(2920, 25, 53)"
Chunk shape,"(100, 25, 53)"
Order,C
Read-only,False
Compressor,"Blosc(cname='lz4', clevel=5, shuffle=SHUFFLE, blocksize=0)"
Store type,zarr.storage.DirectoryStore
No. bytes,15476000 (14.8M)
No. bytes stored,9209491 (8.8M)


In [9]:
from rechunker import rechunk

target_chunks_dict = {'time': 2920, 'lat': 25, 'lon': 1}
max_mem = '1MB'

target_store = 'air_rechunked.zarr'
temp_store = 'air_rechunked-tmp.zarr'

array_plan = rechunk(source_array, target_chunks_dict, max_mem, target_store, temp_store=temp_store)
array_plan

<Rechunked>
* Source      : <zarr.core.Array '/air' (2920, 25, 53) float32>
* Intermediate: <zarr.core.Array (2920, 25, 53) float32>
* Target      : <zarr.core.Array (2920, 25, 53) float32>

In [12]:
from dask.diagnostics import ProgressBar

with ProgressBar():
    result = array_plan.execute()
result.chunks

[########################################] | 100% Completed |  0.7s


(2920, 25, 1)

In [13]:
target_array = zarr.open('air_rechunked.zarr')
target_array

<zarr.core.Array (2920, 25, 53) float32>

In [14]:
target_array_dask = dsa.from_zarr('air_rechunked.zarr')
target_array_dask

,Array,Chunk
Bytes,15.48 MB,292.00 kB
Shape,"(2920, 25, 53)","(2920, 25, 1)"
Count,54 Tasks,53 Chunks
Type,float32,numpy.ndarray


In [15]:
target_chunks = {
    'air': {'time': 2920, 'lat': 25, 'lon': 1},
    'time': None, # don't rechunk this array
    'lon': None,
    'lat': None,
}
max_mem = '1MB'

target_store = 'group_rechunked.zarr'
temp_store = 'group_rechunked-tmp.zarr'

array_plan = rechunk(source_group, target_chunks, max_mem, target_store, temp_store=temp_store)
array_plan

<Rechunked>
* Source      : <zarr.hierarchy.Group '/'>
* Intermediate: <zarr.hierarchy.Group '/'>
* Target      : <zarr.hierarchy.Group '/'>

In [17]:
array_plan.execute()

<zarr.hierarchy.Group '/'>

In [18]:
xr.open_zarr('group_rechunked.zarr')

<xarray.Dataset>
Dimensions:  (lat: 25, lon: 53, time: 2920)
Coordinates:
  * lat      (lat) float32 75.0 72.5 70.0 67.5 65.0 ... 25.0 22.5 20.0 17.5 15.0
  * lon      (lon) float32 200.0 202.5 205.0 207.5 ... 322.5 325.0 327.5 330.0
  * time     (time) datetime64[ns] 2013-01-01 ... 2014-12-31T18:00:00
Data variables:
    air      (time, lat, lon) float32 dask.array<chunksize=(2920, 25, 1), meta=np.ndarray>
Attributes:
    Conventions:  COARDS
    description:  Data is from NMC initialized reanalysis\n(4x/day).  These a...
    platform:     Model
    references:   http://www.esrl.noaa.gov/psd/data/gridded/data.ncep.reanaly...
    title:        4x daily NMC reanalysis (1948)